## Add noise to image

In [13]:
import numpy as np
from PIL import Image
import os, sys
import multiprocessing
from multiprocessing import Pool
import glob
import fnmatch
import shutil
import matplotlib.pyplot as plt
import cv2
from skimage.util import random_noise as rdn

ImportError: No module named pathos.multiprocessing

In [11]:
def read_images(path, sz=None): 
    '''
    Chargement des données
    '''
    c=0
    X,y = [], []
    for dirname , dirnames , filenames in os.walk(path):
        for subdirname in dirnames:
            subject_path = os.path.join(dirname , subdirname) 
            for filename in os.listdir(subject_path):
                try:
                    im = Image.open(os.path.join(subject_path , filename)) 
                    im = im.convert("L")
                    # resize to given size (if given)
                    if (sz is not None):
                        im = im.resize(sz, Image.ANTIALIAS)     
                    X.append(np.asarray(im, dtype=np.uint8)) 
                    y.append(c)
                except IOError:
                    print("I/O error({0}): {1}".format("errno", "strerror"))
                except:
                    print("Unexpected error:", sys.exc_info()[0] )
                    raise
            c = c+1 
    return [X,y]

In [22]:
def gaussian_noise(im_name,amount, sigma ):
    #amount = 0.5
    #sigma = 15
    pil_image = Image.open(im_name)

    # Convert to luminance
    im_size = pil_image.size
    ## EDIT HERE
    im_array = np.asarray(pil_image).astype(float)
    vector=im_array.ravel().copy()
    totalSize=im_size[0]*im_size[1]

    index=np.random.randint(totalSize,size=int(totalSize*amount))
    noise=np.random.randn(1,int(totalSize*amount))*sigma

    noisy_vector=vector
    noisy_vector.flags.writeable = True
    noisy_vector[index]=noisy_vector[index]+noise
    noisy_img=noisy_vector.reshape((im_size[1],im_size[0]))
    noisy_img = np.round(noisy_img)
    noisy_img[noisy_img<0]=0
    noisy_img[noisy_img>=255]=255

    # create appropriate dirrectorries
    dir_model = os.path.split(os.path.split(im_name)[0])[0]
    dir_model = dir_model.replace(source_root,process_root)
    #print dir_model
    if not os.path.exists(dir_model):
        os.makedirs(dir_model)
    dir_block = os.path.split(im_name)[0]
    dir_block = dir_block.replace(source_root,process_root)
    #print dir_block
    if not os.path.exists(dir_block):
        os.makedirs(dir_block)
    im_out_name = im_name.replace(source_root,process_root)
    noisy_img = noisy_img.astype(dtype=np.uint8)
    im_noise_pgm = Image.fromarray(noisy_img)
    im_noise_pgm.save(im_out_name)


## On Yale Database

In [7]:
## Change path to images
source_root = '/home/ldarmet/Face recognition/CroppedYale'
process_root = '/home/ldarmet/Face recognition/CroppedYaleNoisy' #Create noisy images in a new directories

In [8]:
list_im=[]
for root, dirnames, filenames in os.walk(source_root):
    for filename in fnmatch.filter(filenames, '*.pgm'):
        list_im.append(os.path.join(root, filename))

In [9]:
list_im

['/home/ldarmet/Face recognition/CroppedYale/yaleB02/yaleB02_P00A-085E-20.pgm',
 '/home/ldarmet/Face recognition/CroppedYale/yaleB02/yaleB02_P00_Ambient.pgm',
 '/home/ldarmet/Face recognition/CroppedYale/yaleB02/yaleB02_P00A-070E+45.pgm',
 '/home/ldarmet/Face recognition/CroppedYale/yaleB02/yaleB02_P00A+060E+20.pgm',
 '/home/ldarmet/Face recognition/CroppedYale/yaleB02/yaleB02_P00A+005E+10.pgm',
 '/home/ldarmet/Face recognition/CroppedYale/yaleB02/yaleB02_P00A-020E-40.pgm',
 '/home/ldarmet/Face recognition/CroppedYale/yaleB02/yaleB02_P00A-070E+00.pgm',
 '/home/ldarmet/Face recognition/CroppedYale/yaleB02/yaleB02_P00A-005E-10.pgm',
 '/home/ldarmet/Face recognition/CroppedYale/yaleB02/yaleB02_P00A+070E-35.pgm',
 '/home/ldarmet/Face recognition/CroppedYale/yaleB02/yaleB02_P00A-020E+10.pgm',
 '/home/ldarmet/Face recognition/CroppedYale/yaleB02/yaleB02_P00A-130E+20.pgm',
 '/home/ldarmet/Face recognition/CroppedYale/yaleB02/yaleB02_P00A+110E+15.pgm',
 '/home/ldarmet/Face recognition/CroppedY

In [27]:
import functools

if not os.path.exists(process_root):
    os.makedirs(process_root)    
nbCores = multiprocessing.cpu_count()
pool = Pool(nbCores)
pool.map(functools.partial(gaussian_noise,amount=0.5, sigma=0.15), list_im)
pool.close()
pool.join()